In [1]:
# импорт библиотек
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [2]:
# загружаем датафрейм с анекдотами
df_original = pd.read_csv('posts.csv').set_index('id')
df_original

,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
3,"— Вы уверены, что хотите выключить компьютер? ..."
4,"Извините, но подписчик из меня так себе"
...,...
24651,Марк Цукерберг и Павел Дуров заходят в бар. Цу...
24652,"— Вы любите Кафку? — Да, грефневую!"
24653,На распродаже человеческих органов нaчалась да...


Некоторые матерные слова зацензурены символами * или # что усложняет обработку этих слов. Анекдотов содержащих данные символы немного, поэтому можно их и удалить. Также уберем все анекдоты содержащие английские символы.

In [3]:
df = df_original[~df_original.text.str.contains('[\*#\[\]a-zA-Z]')]
df.drop(df[df.text.str.len() < 70].index, inplace=True)
df

C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text
id,
0,"Стоят два наркомана на перроне, мимо проезжает..."
1,В советские времена объявили месяц вежливого о...
2,Полицейские повязали целую группу проституток ...
5,"— Дед, почему наше село называется Астафьево? ..."
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ..."
...,...
24646,"Пожар в Техасе, в школе: - Кидай мне детей, я ..."
24647,Боксёры не занимаются сексом перед боем. Потом...
24648,Судья на заседании так долго призывал к порядк...


In [4]:
# для токенизации используем стеммер Портера
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) \
                   for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-ЯёЁ]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [5]:
# переводим датафрейм в словарь с текстом анекдота и с листом токенов
df_dict_duplicates = dict() 
for i, text in tqdm(df['text'].iteritems()):
    df_dict_duplicates[i] = [text, token_and_stem(text)]

# удаляем из словаря повторяющиеся анекдоты (те у кого один и тот же лист токенов)
temp = set()
df_dict = dict()
for key, value in df_dict_duplicates.items():
    if tuple(set(value[1])) not in temp:
        temp.add(tuple(set(value[1])))
        df_dict[key] = value

print("Удалено",  len(df_dict_duplicates) - len(df_dict), "дубликатов")

15583it [00:46, 332.72it/s]


Удалено 835 дубликатов


In [6]:
def dummy_fun(doc):
    return doc

stopwords = nltk.corpus.stopwords.words('russian')

# расширяем список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

# создаем лист с токенами всех анекдотов
tokenized = [value[1] for value in tqdm(df_dict.values())]

max_df = 0.6
min_df = 1e-3
max_features = 100000
ngram_range = (1, 3)
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, max_features=max_features,
                                   min_df=min_df, stop_words=stopwords,
                                   tokenizer=dummy_fun, preprocessor=dummy_fun,
                                   use_idf=True, ngram_range=ngram_range)

tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized)
tfidf_matrix

100%|██████████| 14748/14748 [00:00<00:00, 3684631.61it/s]
C:\Users\steel\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['б', 'г', 'д', 'е', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'х', 'ц', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю'] not in stop_words.
  warnings.warn(


<14748x4634 sparse matrix of type '<class 'numpy.float64'>'
	with 329126 stored elements in Compressed Sparse Row format>

In [7]:
# экспериментальном путем выведено, что оптимальное количество кластеров ~200
n_clusters = 200

agglo = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean')
answer = agglo.fit_predict(tfidf_matrix.toarray())
answer

array([ 4,  4,  5, ...,  4,  4, 66], dtype=int64)

In [8]:
i = 0
for key, value in df_dict.items():
    df_dict[key].append(answer[i])
    i += 1

df = pd.DataFrame.from_dict(df_dict, orient='index')
df.to_csv('clustered_posts.csv')
df

,0,1,2
0,"Стоят два наркомана на перроне, мимо проезжает...","[сто, два, наркома, на, перрон, мим, проезжа, ...",4
1,В советские времена объявили месяц вежливого о...,"[в, советск, врем, объяв, месяц, вежлив, обслу...",4
2,Полицейские повязали целую группу проституток ...,"[полицейск, повяза, цел, групп, проституток, с...",5
5,"— Дед, почему наше село называется Астафьево? ...","[дед, поч, наш, сел, называ, астафьев, дык, на...",157
8,"— Алло, здравствуйте, это Ремонт Ноутбуков? — ...","[алл, здравств, эт, ремонт, ноутбук, да, здрав...",45
...,...,...,...
24622,- Откуда у тебя топор? - Мне Королева эльфов ...,"[откуд, у, теб, топор, мне, королев, эльф, дал...",4
24632,— Официант! Почему у меня кофе воняет хуями? —...,"[официант, поч, у, мен, коф, воня, ху, а, вы, ...",40
24637,— Давай поедем на выходные к океану. — К како...,"[дава, поед, на, выходн, к, океан, к, как, оке...",4
24639,"— Девушка, как вы прекрасны в этом вечернем ту...","[девушк, как, вы, прекрасн, в, эт, вечерн, туа...",4


In [12]:
pd.set_option('display.max_colwidth', 0) 
df[df[2] == 122] # анекдоты про священников

,0,1,2
214,"Скажите, святой отец, а почему в католической церкви хор поет под клавесин, орган или фисгармонию, а у нас, православных — без аккомпанемента? — Дело в том, сын мой, что настоящий талант не пропьешь. А вот клавесин — как нехуй делать.","[скаж, свят, отец, а, поч, в, католическ, церкв, хор, поет, под, клавесин, орга, ил, фисгармон, а, у, нас, православн, без, аккомпанемент, дел, в, том, сын, мо, что, настоя, талант, не, пропьеш, а, вот, клавесин, как, нех, дела]",122
1285,"— Простите, святой отец, ибо я согрешил. В прошлую пятницу я переспал с мужчиной. — И не перезвонил.","[прост, свят, отец, иб, я, согреш, в, прошл, пятниц, я, переспа, с, мужчин, и, не, перезвон]",122
1687,"- Скажите, батюшка, а почему в католической церкви хор поет под клавесин, орган или фисгармонию, а у нас, православных - без аккомпанемента? - Дело в том, матушка, что настоящий талант не пропьешь. А вот клавесин - как нех…й делать.","[скаж, батюшк, а, поч, в, католическ, церкв, хор, поет, под, клавесин, орга, ил, фисгармон, а, у, нас, православн, без, аккомпанемент, дел, в, том, матушк, что, настоя, талант, не, пропьеш, а, вот, клавесин, как, нех…, дела]",122
3196,"Старый священник уходит на покой. На его место назначили молодого. В первый же день приходит к нему исповедоваться мужичок: — Святой отец, грешен я. — В чем грех твой? - вопрошает священник. — Грех мой - мужеложство, святой отец. Ну и тут поп с ужасом понимает, что не знает что назначить мужичку во искупление греха. Сколько раз помолится грешному? Сколько свечек поставить и кому?? Не знает и все тут! Выбегает он в церковный двор, а там мальчишка-послушник метлой машет. Ну и поп к нему: — Отрок! Не помнишь ли ты, что прежний батюшка давал за мужеложство?? — Ну, когда пирожок, а когда и яблочко.","[стар, священник, уход, на, пок, на, ег, мест, назнач, молод, в, перв, же, ден, приход, к, нем, исповедова, мужичок, свят, отец, греш, я, в, чем, грех, тво, вопроша, священник, грех, мо, мужеложств, свят, отец, ну, и, тут, поп, с, ужас, понима, что, не, знает, что, назнач, мужичк, во, искуплен, грех, скольк, раз, помол, грешн, скольк, свечек, постав, и, ком, не, знает, и, все, тут, выбега, он, в, церковн, двор, а, там, мальчишка-послушник, метл, машет, ну, и, поп, к, нем, отрок, не, помн, ли, ты, что, прежн, батюшк, дава, за, мужеложств, ну, когд, пирожок, а, когд, и, яблочк]",122
3290,"Изобрёл Попов радио, включил, а оттуда дымок пошёл. "" Ничего, сейчас покурят, да что-нибудь скажут,"" - подумал Попов.","[изобрел, поп, рад, включ, а, оттуд, дымок, пошел., нич, сейчас, покур, да, что-нибуд, скажут, подума, поп]",122
3967,"Похороны. Молодая женщина хоронит своего мужа. Вдова в глубоком трауре, глаза, красные от слёз. Священник читает молитву и произносит слова утешения родным и близким покойног. -Так устроен этот мир, - говорит святой отец. - Все мы рано или поздно отойдём в мир иной. Смерть - это тайна, но в тайне есть дверь. Сегодня ваш дорогой сын, муж и отец приоткрыл эту дверь, он ушёл от нас, но он навеки останется в наших сердцах. Мы всегда будем помнить его светлый образ... И когда вам будет особенно тяжело... (обращается к молодой вдове) вспоминайте, как любили этого человека, как тепло и радостно вам было рядом с ним. Вспоминайте его лицо, его руки, его последние слова... Вы помните его последние слова? -Да, святой отец. -И что же он сказал? -Из этого ружья, корова, ты даже в слона не попадёшь!","[похорон, молод, женщин, хорон, сво, муж, вдов, в, глубок, траур, глаз, красн, от, слез, священник, чита, молитв, и, произнос, слов, утешен, родн, и, близк, покойног, -так, устро, этот, мир, говор, свят, отец, все, мы, ран, ил, поздн, отойд, в, мир, ин, смерт, эт, тайн, но, в, тайн, ест, двер, сегодн, ваш, дорог, сын, муж, и, отец, приоткр, эт, двер, он, ушел, от, нас, но, он, навек, останет, в, наш, сердц, мы, всегд, буд, помн, ег, светл, образ, и, когд, вам, будет, особен, тяжел, обраща, к, молод, вдов, вспомина, как, люб, эт, человек, как, тепл, и, радостн, вам, был, ряд, с, ...]",122
8